## Lấy dữ liệu đã được xử lí từ file có sẵn

In [ ]:
import pandas as pd
data = pd.read_csv('newdfBDS.csv')
print(len(data))
data.head(2)

## Lọc để chỉ làm việc với dữ liệu tháng 8 - 12/2020

Thời điểm xây dựng mô hình là tháng 1/2021 nên tác giả chỉ lấy dữ liệu trong khoảng thời gian gần đó. Bạn đọc có thể xem xét dữ liệu thuộc các khoảng thời gian khác để đánh giá thêm

In [ ]:
data_aug = data.loc[data['month'] == 8]
data_sep = data.loc[data['month'] == 9]
data_oct = data.loc[data['month'] == 10]
data_nov = data.loc[data['month'] == 11]
data_dec = data.loc[data['month'] == 12]
data_new = pd.concat([data_aug, data_sep, data_oct, data_nov, data_dec])
print('So luong ban ghi thang 8 - 12: ', len(data_new))
data_new.head(3)

#### Một vài bước tiền xử lí dữ liệu trước khi huấn luyện mô hình

In [ ]:
data_new.describe()

In [ ]:
#Chuyen gia tri truong 'month' thanh dang string (dạng categorical)
data_new = data_new.astype({"month": str})

In [ ]:
data_new.head()

In [ ]:
#Xử lí các trường dạng số - Chuẩn hóa giá trị
from sklearn.preprocessing import MinMaxScaler
mmscaler = MinMaxScaler()
X_num = data[['square', 'bedrooms', 'bathrooms']]
mmscaler.fit(X_num)

X_num = mmscaler.transform(X_num)

In [ ]:
##Lọc lấy các trường category để xử lí :">> (Đưa về các biến giả dummy)
X_cat = data_new.drop(['square', 'bedrooms', 'bathrooms', 'price', 'id'], axis=1)
X_cat = X_cat[['project', 'investor', 'district', 'ward']]

import pandas as pd
X_cat_new = pd.get_dummies(data = X_cat)
X_cat_new.head()

In [ ]:
#X là ma trận chứa thông tin các trường thuộc tính
#y là vecto giá tương ứng

import numpy as np
X = np.concatenate([X_num, X_cat_new], axis=1)
y = data_new[['price']].to_numpy()

In [ ]:
#Lưu lại X, y đã xử lý để phòng sự số xảy ra
import pickle as pkl
pkl.dump(X, open('X.pkl', 'wb'))
pkl.dump(y, open('y.pkl', 'wb'))

In [ ]:
#Khi cần, load lại X, y từ file, tương ứng là ma trận thuộc tính và vecto nhãn của dữ liệu.

import pickle as pkl
X = pkl.load(open('X.pkl', 'rb'))
y = pkl.load(open('y.pkl', 'rb')).ravel()

In [ ]:
 print(X.shape)
 print(y.shape)

In [ ]:
#Chia train/test
x_train , x_test , y_train , y_test = train_test_split(X , y , test_size = 0.5,random_state =2, shuffle = True)

## Xây dựng mô hình

### Ham danh gia mo hinh tren du lieu tap train va tap test

In [ ]:


from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def evaluate(model, x_train, x_test, y_train, y_test):

  y_pred =  model.predict(x_test)
  r2 = r2_score(y_test, y_pred) ### Tap test
  mse = mean_squared_error(y_test, y_pred)
  mae =  mean_absolute_error(y_test,  y_pred)
  print('r2: ', r2, '\nmse: ', mse, '\nmae: ', mae)
  print('------------------------------------------')

  y_train_pred = model.predict(x_train) ### Tap train
  r2 = r2_score(y_train, y_train_pred)
  mse = mean_squared_error(y_train, y_train_pred)
  mae =  mean_absolute_error(y_train, y_train_pred)
  print('r2: ', r2, '\nmse: ', mse, '\nmae: ', mae)

  print(type(y_train))
  print(type(y_train_pred))

### Khởi tạo mô hình và huấn luyện:

In [ ]:
# Mô hình với các siêu tham số khởi đầu
from sklearn import ensemble
clf = ensemble.GradientBoostingRegressor(n_estimators = 400, max_depth = 5, min_samples_split = 2,
          learning_rate = 0.1, loss = 'ls')

In [ ]:
clf.fit(x_train, y_train)

In [ ]:
evaluate(clf, x_train, x_test, y_train, y_test)

### Tối ưu mô hình bằng việc tinh chỉnh (tunning) siêu tham số




In [ ]:
#Liet ke các bo gia tri de tunning cho cac sieu tham so tuong ung cua mo hinh

param = {
         'max_depth':[2,3,4,5,6,7],
         'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001],
         'n_estimators':[100, 150,200,500,900,1200,1500],
         'n_estimators':[100,250,500,750,1000,1250,1500,1750],
         'min_samples_split':[2,4,6,8,10,20,40,60,100],
         'min_samples_leaf':[1,3,5,7,9],
         'max_features':[2,3,4,5,6,7],
         'subsample':[0.7,0.75,0.8,0.85,0.9,0.95,1]
         }

In [ ]:
#thực hiện cross-validation để chọn bộ siêu tham số
from sklearn.model_selection import RandomizedSearchCV
random_cv = RandomizedSearchCV(estimator=clf,
                               param_distributions=param,
                               cv=5,n_iter=50,
                               scoring='neg_mean_absolute_error',n_jobs=4,
                               verbose=5,
                               return_train_score=True,
                               random_state=42)

In [ ]:
random_cv.fit(x_train,y_train)

In [ ]:
#Lấy ra bộ tham số sau khi thực hiện CV (cross-validation)
random_cv.best_estimator_

In [ ]:
#Xây dựng mô hình với bộ tham số vừa tìm được
clf2 = ensemble.GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.15, loss='ls', max_depth=6,
                          max_features=5, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=4,
                          min_weight_fraction_leaf=0.0, n_estimators=1750,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=0.85, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
clf2.fit(x_train, y_train)

In [ ]:
evaluate(clf2, x_train, x_test, y_train, y_test)

In [ ]:
#Lưu mô hình

import pickle

pickle.dump(clf2, open('clf2.pkl', 'wb'))

In [ ]:
model2 = pickle.load(open('clf2.pkl', 'rb'))